In [5]:
from __future__ import division
import numpy as np
import scipy as sp
from ttide.t_astron import t_astron
from ttide.t_getconsts import t_getconsts
import ttide.t_utils as tu

In [4]:
def t_vuf(ltype, ctime, ju, lat=None):

    astro, ader = t_astron(ctime)

    if ltype == "full":
        const = t_getconsts(ctime)
        # Phase relative to Greenwich (in units of cycles).
        v = np.fmod(np.dot(const.doodson, astro) + const.semi, 1)
        v = v[(ju - 1)]
        u = np.zeros(shape=(v.shape, v.shape), dtype="float64")
        f = np.ones(shape=(v.shape, v.shape), dtype="float64")
    else:
        const, sat, shallow = t_getconsts(ctime)

        v = np.fmod(np.dot(const["doodson"], astro) + const["semi"], 1)

        if lat is not None:

            if abs(lat) < 5:
                lat = np.sign(lat) * 5
            slat = np.sin(np.pi * lat / 180)
            # Satellite amplitude ratio adjustment for latitude.
            rr = sat["amprat"]
            # no amplitude correction
            if np.isfinite(lat):
                j = np.flatnonzero(sat["ilatfac"] == 1)
                # latitude correction for diurnal constituents
                rr[j] = rr[j] * 0.36309 * (1.0 - 5.0 * slat * slat) / slat
                j = np.flatnonzero(sat["ilatfac"] == 2)
                # latitude correction for semi-diurnal constituents
                rr[j] = rr[j] * 2.59808 * slat
            else:
                rr[sat["ilatfac"] > 0] = 0
            # Calculate nodal amplitude and phase corrections.
            uu = np.fmod(np.dot(sat["deldood"], astro.T[3:6]) + sat["phcorr"], 1)

            nsat = np.max(sat["iconst"].shape)
            nfreq = np.max(const["isat"].shape)

            fsum = np.array(
                1
                + sp.sparse.csr_matrix(
                    (
                        np.squeeze(rr * np.exp(1j * 2 * np.pi * uu)),
                        (np.arange(0, nsat), np.squeeze(sat["iconst"] - 1)),
                    ),
                    shape=(nsat, nfreq),
                ).sum(axis=0)
            ).flatten()

            f = np.absolute(fsum)
            u = np.angle(fsum) / (2 * np.pi)

            # Compute amplitude and phase corrections
            # for shallow water constituents.
            shallow_m1 = const["ishallow"].astype(int) - 1
            iname_m1 = shallow["iname"].astype(int) - 1
            coefs = shallow["coef"].astype(np.float64)
            range_cache = {n: np.arange(n) for n in range(const["nshallow"].max() + 1)}
            for k in np.flatnonzero(np.isfinite(const["ishallow"])):
                ik = shallow_m1[k] + range_cache[const["nshallow"][k]]
                iname = iname_m1[ik]
                coef = coefs[ik]
                f[k] = np.multiply.reduce(np.power(f[iname], coef))
                u[k] = u[iname].dot(coef)
                v[k] = v[iname].dot(coef)

            f = f[ju]
            u = u[ju]
            v = v[ju]

        else:

            for k in np.flatnonzero(np.isfinite(const["ishallow"])):
                ik = (
                    const["ishallow"][k] - 1 + np.array(range(0, const["nshallow"][k]))
                ).astype(int)
                v[k] = np.dot(v[shallow["iname"][ik] - 1], shallow["coef"][ik])

            v = v[ju]
            f = np.ones(len(v))
            u = np.zeros(len(v))

    return v, u, f



In [6]:
a = t_vuf(ltype="nodal", ctime=0, ju=1, lat=0)")